In [1]:
!pip install pgeocode


tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [2]:
import pgeocode
import pandas as pd
import requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [3]:
nomi = pgeocode.Nominatim('in')

In [4]:
nomi.query_postal_code("560072")

postal_code                                560072
country code                                   IN
place_name        Nagarbhavi, Nagarbhavi II Stage
state_name                              Karnataka
state_code                                     19
county_name                             Bengaluru
county_code                                   583
community_name                    Bangalore North
community_code                                NaN
latitude                                  13.2257
longitude                                 77.5431
accuracy                                        4
Name: 0, dtype: object

In [5]:
India = nomi._get_data('in')

In [6]:
india = nomi._index_postal_codes()

In [7]:
karnataka = india[india.state_name == "Karnataka"]

In [8]:
bengaluru = karnataka[karnataka.county_name == "Bengaluru"]

In [9]:
bengaluru.head()

,country code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
10165,IN,560001,"Vasanthanagar, Mahatma Gandhi Road, Legislator...",Karnataka,19,Bengaluru,583.0,Bangalore North,NaN,13.2257,77.592244,4
10166,IN,560002,"Bangalore City, Bangalore Corporation Building",Karnataka,19,Bengaluru,583.0,Bangalore North,NaN,13.2257,77.584350,4
10167,IN,560003,"Malleswaram, Palace Guttahalli, Vyalikaval Ext...",Karnataka,19,Bengaluru,583.0,Bangalore North,NaN,13.2257,77.567350,4
10168,IN,560004,"Basavanagudi H.O, Mavalli, Pampamahakavi Road,...",Karnataka,19,Bengaluru,583.0,Bangalore South,NaN,12.9833,77.583300,1
10169,IN,560005,Fraser Town,Karnataka,19,Bengaluru,583.0,Bangalore North,NaN,12.9910,77.584300,1


In [10]:
bengaluru.shape

(107, 12)

In [11]:
bengaluru.reset_index(inplace = True)

In [12]:
bengaluru.columns

Index(['index', 'country code', 'postal_code', 'place_name', 'state_name',
       'state_code', 'county_name', 'county_code', 'community_name',
       'community_code', 'latitude', 'longitude', 'accuracy'],
      dtype='object')

In [13]:
bengaluru = bengaluru[['postal_code','place_name', 'community_name', 'latitude', 'longitude']]

In [14]:
bengaluru.head(10)

,postal_code,place_name,community_name,latitude,longitude
0,560001,"Vasanthanagar, Mahatma Gandhi Road, Legislator...",Bangalore North,13.2257,77.592244
1,560002,"Bangalore City, Bangalore Corporation Building",Bangalore North,13.2257,77.584350
2,560003,"Malleswaram, Palace Guttahalli, Vyalikaval Ext...",Bangalore North,13.2257,77.567350
3,560004,"Basavanagudi H.O, Mavalli, Pampamahakavi Road,...",Bangalore South,12.9833,77.583300
4,560005,Fraser Town,Bangalore North,12.9910,77.584300
5,560006,"Training Command IAF, J.C.Nagar",Bangalore North,12.9910,77.584300
6,560007,"Air Force Hospital, Agram",Bangalore North,12.9910,77.584300
7,560008,"Hulsur Bazaar, H.A.L II Stage H.O",Bangalore North,12.9910,77.584300
8,560009,"K. G. Road, Bangalore Dist Offices Bldg",Bangalore North,12.9910,77.584300
9,560010,"Rajajinagar IVth Block, Rajajinagar H.O, Indus...",Bangalore North,12.9604,77.567300


In [15]:
bengaluru.community_name.unique()

array(['Bangalore North', 'Bangalore South', 'Bengaluru', 'Bangalore',
       'Bellandur', 'Anekal', 'Hosakote', nan, 'Nla & Bgsouth', 'Bg North'], dtype=object)

In [16]:
# 'Banglorenorth' ,'Bg North' to Bangalore North
# 'Bangaloresouth' to 'Bangalore South'
bengaluru.loc[bengaluru.community_name == 'Bangaloresouth', 'community_name'] = 'Bangalore South'
bengaluru.loc[bengaluru.community_name == 'Bangalorenorth', 'community_name'] = 'Bangalore North'
bengaluru.loc[bengaluru.community_name == 'Bg North', 'community_name'] = 'Bangalore North'


In [17]:
bengaluru.columns

Index(['postal_code', 'place_name', 'community_name', 'latitude', 'longitude'], dtype='object')

In [18]:

address = 'Bengaluru, KA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru are 12.9791198, 77.5912997.


In [19]:
# create map of Bengaluru using latitude and longitude values
map_Bengaluru = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, place, community in zip(bengaluru['latitude'], bengaluru['longitude'], bengaluru['place_name'], bengaluru['community_name']):
    label = '{}, {}'.format(community, place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bengaluru)  
    
map_Bengaluru

In [20]:
bengaluru.groupby("community_name").count()

,postal_code,place_name,latitude,longitude
community_name,,,,
Anekal,4,4,4,4
Bangalore,8,8,8,8
Bangalore North,61,61,61,61
Bangalore South,27,27,27,27
Bellandur,1,1,1,1
Bengaluru,2,2,2,2
Hosakote,2,2,2,2
Nla & Bgsouth,1,1,1,1


In [21]:
#as Bangalore north has highest count, will be using for further clustering

In [22]:
blr_north = bengaluru[bengaluru['community_name'] == 'Bangalore North'].reset_index(drop=True)

In [23]:
blr_north.head()

,postal_code,place_name,community_name,latitude,longitude
0,560001,"Vasanthanagar, Mahatma Gandhi Road, Legislator...",Bangalore North,13.2257,77.592244
1,560002,"Bangalore City, Bangalore Corporation Building",Bangalore North,13.2257,77.584350
2,560003,"Malleswaram, Palace Guttahalli, Vyalikaval Ext...",Bangalore North,13.2257,77.567350
3,560005,Fraser Town,Bangalore North,12.9910,77.584300
4,560006,"Training Command IAF, J.C.Nagar",Bangalore North,12.9910,77.584300


In [24]:
#Lets plot Bangalore North Map
address = 'Bangalore North,KA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore North are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore North are 12.9791198, 77.5912997.


In [25]:
# create map of Bengaluru using latitude and longitude values
map_blrNorth = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, place, community in zip(blr_north['latitude'], blr_north['longitude'], blr_north['place_name'], blr_north['community_name']):
    label = '{}, {}'.format(community, place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_blrNorth)  
    
map_blrNorth

In [26]:
CLIENT_ID = 'LXRTQ4XM1403FUBFBJC5FV5IR4AYGWK4KNMBDC3ITQNW0XNX' # your Foursquare ID
CLIENT_SECRET = 'BRD5IUBAEQ55J3WPIJHSSP3CURLSYUURJKMA3BVSLUES0X4B' # your Foursquare Secret
VERSION = '20190131' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LXRTQ4XM1403FUBFBJC5FV5IR4AYGWK4KNMBDC3ITQNW0XNX
CLIENT_SECRET:BRD5IUBAEQ55J3WPIJHSSP3CURLSYUURJKMA3BVSLUES0X4B


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
blrNorth_venues = getNearbyVenues(names=blr_north['place_name'],
                                   latitudes=blr_north['latitude'],
                                   longitudes=blr_north['longitude']
                                  )

Vasanthanagar, Mahatma Gandhi Road, Legislators Home, Dr. Ambedkar Veedhi, Bangalore G.P.O., Rajbhavan (Bangalore), CMM Court Complex, HighCourt, Vidhana Soudha
Bangalore City, Bangalore Corporation Building
Malleswaram, Palace Guttahalli, Vyalikaval Extn, Swimming Pool Extn
Fraser Town
Training Command IAF, J.C.Nagar
Air Force Hospital, Agram
Hulsur Bazaar, H.A.L II Stage H.O
K. G. Road, Bangalore Dist Offices Bldg
Rajajinagar IVth Block, Rajajinagar H.O, Industrial Estate (Bangalore)
Science Institute
Jalahalli East, Jalahalli H.O
Jalahalli West
Doorvaninagar, Ramamurthy Nagar, Krishnarajapuram R S
Vimanapura, NAL
Seshadripuram
Gayathrinagar, Srirampuram
Yeswanthpura, Yeshwanthpur Bazar
Magadi Road
H.A. Farm, Anandnagar (Bangalore), Hebbal Kempapura
Bangalore Sub Foreign Post, Richmond Town, Museum Road, CMP Centre And School
R T Nagar H.O, P&T Col. Kavalbyrasandra
Maruthi Sevanagar
Krishnarajapuram, Devasandra
Rameshnagar, Doddanekkundi, Marathahalli Colony, Kundalahalli
Indiranagar

In [29]:
blrNorth_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Fraser Town,12.991,77.5843,Bangalore Golf Club,12.989681,77.585933,Golf Course
1,Fraser Town,12.991,77.5843,The Sugar Factory,12.990041,77.586320,Nightclub
2,Fraser Town,12.991,77.5843,Chitra Kala Parishad,12.989295,77.581115,Art Gallery
3,Fraser Town,12.991,77.5843,ITC Windsor,12.994131,77.585896,Hotel
4,Fraser Town,12.991,77.5843,The Raj Pavilion,12.994108,77.585610,Café
5,Fraser Town,12.991,77.5843,Dublin,12.994301,77.585712,Pub
6,Fraser Town,12.991,77.5843,"Shangri-La Hotel, Bengaluru",12.992112,77.588446,Hotel
7,Fraser Town,12.991,77.5843,Raintree,12.994229,77.584316,Art Gallery
8,Fraser Town,12.991,77.5843,Oko @ Lalit Ashok,12.992372,77.582449,Asian Restaurant
9,Fraser Town,12.991,77.5843,Dum Pukht Jolly Nabobs,12.994142,77.586308,Restaurant


In [30]:
#lets figure out how many unique venues category
print('There are {} uniques categories.'.format(len(blrNorth_venues['Venue Category'].unique())))

There are 63 uniques categories.


In [31]:
#Let us cluster based on neighborhoods
# one hot encoding
blrNorth_onehot = pd.get_dummies(blrNorth_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blrNorth_onehot['Neighborhood'] = blrNorth_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [blrNorth_onehot.columns[-1]] + list(blrNorth_onehot.columns[:-1])
blrNorth_onehot = blrNorth_onehot[fixed_columns]

blrNorth_grouped = blrNorth_onehot.groupby('Neighborhood').mean().reset_index()

In [32]:
blrNorth_grouped.head(10)

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Sporting Goods Shop,Sushi Restaurant,Theme Park Ride / Attraction,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"A F Station Yelahanka, BSF Campus Yelahanka",0.333333,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0000,0.0,0.00000
1,"Air Force Hospital, Agram",0.000000,0.0,0.0,0.00000,0.074074,0.037037,0.037037,0.00000,0.00000,...,0.037037,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0000,0.0,0.00000
2,"Bangalore Sub Foreign Post, Richmond Town, Mus...",0.000000,0.0,0.0,0.03125,0.000000,0.000000,0.000000,0.03125,0.03125,...,0.000000,0.0,0.03125,0.0,0.03125,0.03125,0.0,0.0625,0.0,0.03125
3,"Basaveshwaranagar, Kamakshipalya, K H B Colony",0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.2,0.0000,0.0,0.00000
4,Benson Town,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0000,0.0,0.00000
5,C.V.Raman Nagar,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0000,0.0,0.00000
6,Domlur,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0000,0.0,0.00000
7,"Doorvaninagar, Ramamurthy Nagar, Krishnarajapu...",0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0000,0.0,0.00000
8,Fraser Town,0.000000,0.0,0.0,0.00000,0.074074,0.037037,0.037037,0.00000,0.00000,...,0.037037,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0000,0.0,0.00000
9,G.K.V.K.,0.333333,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0000,0.0,0.00000


In [33]:
#Top common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
blrNorth_venues_sorted = pd.DataFrame(columns=columns)
blrNorth_venues_sorted['Neighborhood'] = blrNorth_grouped['Neighborhood']

for ind in np.arange(blrNorth_grouped.shape[0]):
    blrNorth_venues_sorted.iloc[ind, 1:] = return_most_common_venues(blrNorth_grouped.iloc[ind, :], num_top_venues)

blrNorth_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"A F Station Yelahanka, BSF Campus Yelahanka",ATM,Pizza Place,Plaza,Dance Studio,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Department Store,Café
1,"Air Force Hospital, Agram",Indian Restaurant,Hotel,Art Gallery,Café,Nightclub,Coffee Shop,Art Museum,Japanese Restaurant,Italian Restaurant,Event Space
2,"Bangalore Sub Foreign Post, Richmond Town, Mus...",Hotel,Indian Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Japanese Restaurant,Chinese Restaurant,Lounge,Mediterranean Restaurant,Café,Breakfast Spot
3,"Basaveshwaranagar, Kamakshipalya, K H B Colony",Indian Restaurant,Theme Park Ride / Attraction,Hotel Bar,Ice Cream Shop,Chinese Restaurant,Department Store,Cocktail Bar,Coffee Shop,Convenience Store,Dance Studio
4,Benson Town,Farm,Wine Bar,Chinese Restaurant,Fish Market,Fast Food Restaurant,Event Space,Electronics Store,Donut Shop,Dessert Shop,Department Store


In [35]:
# set number of clusters
kclusters = 20

blrNorth_grouped_clustering = blrNorth_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(blrNorth_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:24] 

array([ 0, 18, 12,  8,  1,  4,  1, 14, 18,  0, 10,  2,  9,  1, 18,  1, 14,
       14, 18,  9,  1,  6,  9,  3], dtype=int32)

In [36]:
# add clustering labels
blrNorth_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

blrNorth_merged = blr_north

blrNorth_merged = blrNorth_merged.join(blrNorth_venues_sorted.set_index('Neighborhood'), on='place_name')

blrNorth_merged.head() # check the last c

,postal_code,place_name,community_name,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,560001,"Vasanthanagar, Mahatma Gandhi Road, Legislator...",Bangalore North,13.2257,77.592244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,560002,"Bangalore City, Bangalore Corporation Building",Bangalore North,13.2257,77.584350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,560003,"Malleswaram, Palace Guttahalli, Vyalikaval Ext...",Bangalore North,13.2257,77.567350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,560005,Fraser Town,Bangalore North,12.9910,77.584300,18.0,Indian Restaurant,Hotel,Art Gallery,Café,Nightclub,Coffee Shop,Art Museum,Japanese Restaurant,Italian Restaurant,Event Space
4,560006,"Training Command IAF, J.C.Nagar",Bangalore North,12.9910,77.584300,18.0,Indian Restaurant,Hotel,Art Gallery,Café,Nightclub,Coffee Shop,Art Museum,Japanese Restaurant,Italian Restaurant,Event Space


In [37]:
blrNorth_merged['Cluster Labels'].replace(np.NaN, 0, inplace = True)
blrNorth_merged['Cluster Labels']=blrNorth_merged['Cluster Labels'].astype('int')

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blrNorth_merged['latitude'], blrNorth_merged['longitude'], blrNorth_merged['community_name'], blrNorth_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters